In [ ]:
SERVER_ACCOUNT_KEY_PATH = "config/service_account_cred.json"
SECRETS_OAUTH_2_FILE_PATH = "config/client_secret.json"

BUCKET_API = "martech-sandbox"
GCS_CONNECT_FILE_PATH = "katia/utils/gcs_connect_sandbox_katia.py"
CLIENT_CONFIG_DATA = "clients/001/input_file.xlsx" 
TEMPLATE_PROPERTY_ID = '360045070'
account_id = '6140969720'
container_id = '138007392'
workspace_id = '2'

In [ ]:
# import python libs
from google.cloud import storage
from google.oauth2.service_account import Credentials
import pandas as pd

def create_gcs_connection():
    creds = Credentials.from_service_account_file(SERVER_ACCOUNT_KEY_PATH)
    client = storage.Client(credentials=creds)
    get_gcs_bucket = client.get_bucket(BUCKET_API)
    gcs_blob = get_gcs_bucket.blob(GCS_CONNECT_FILE_PATH)
    contents_gcs = gcs_blob.download_as_string()
    return contents_gcs

def read_gcs_bucket_files(connect_gcs):
    contents = [
        connect_gcs.read_backend_api_method_gtm(),
        connect_gcs.read_api_request(),
        connect_gcs.read_backend_api_utils(),
        connect_gcs.read_backend_api_settings(),
    ]
    return contents


if __name__ == "__main__":

    ####### READ BACKEND FILES #####
    exec(create_gcs_connection())
    print("✅ GCS connected ✅")
    
    connect_gcs = GoogleCloudeStorage(SERVER_ACCOUNT_KEY_PATH)
    backend_contents = read_gcs_bucket_files(connect_gcs)
    
    for content in backend_contents:
        exec(content)

    backend_client_config = connect_gcs.read_client_config(CLIENT_CONFIG_DATA)
    print("✅ Backend files read ✅")
    
    ####### READ CONFIG VARIABLES FOR API SETUP #####
    client_id = settings_api.get("client_id")
    client_secret = settings_api.get("clent_secret")
    api_key = settings_api.get("api_key")

    ####### SETUP GOOGLE ANALYTICS API #####
    df_ga4_propety_ids = pd.read_excel(backend_client_config, sheet_name="Property")
    property_id_list = df_ga4_propety_ids["property_id"].tolist()

    ga4_oper = GoogleAnalyticsOperator(
        client_id, client_secret, SECRETS_OAUTH_2_FILE_PATH, api_key
    )
    access_token = ga4_oper.create_access_token()
    

In [ ]:
ga4_oper.set_api_action()
    
template_api_url = ga4_oper.get_full_api_url(account_id, container_id, workspace_id)
output = read_api_obj(template_api_url, access_token)


In [ ]:
target_item_list = create_api_input_data(backend_client_config, output, ga4_oper.api_action)

    ####### RUN GOOGLE ANALYTICS API#####
for property_id in property_id_list:
    full_api_url = ga4_oper.get_full_api_url(property_id)
    print(f"PROPERTY_ID:{property_id}")
    create_api_obj(full_api_url, access_token, target_item_list